In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import sys

In [7]:
sys.stdout.write("aaa") # 후속 print가 계속 한줄로 나온다
sys.stdout.write("bbb") 
print("aaa")
print("bbb")

aaabbbaaa
bbb


In [8]:
idx2char = ['h','i','e','l','o']

In [63]:
x_data = [0,1,0,2,3,3] # hihell
one_hot_lookup = [[1,0,0,0,0], 
                [0,1,0,0,0],
                [0,0,1,0,0],   
                [0,0,0,1,0],   
                [0,0,0,0,1]]
y_data = [1,0,2,3,3,4] # ihello

x_one_hot = [one_hot_lookup[x] for x in x_data]

print(x_one_hot)

[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0]]


In [108]:
inputs = Variable(torch.Tensor(x_one_hot))# requires_grad=True
labels = Variable(torch.LongTensor(y_data))
test_labels = torch.LongTensor(6,5)
print(test_labels.size())
print(inputs.size())
print(labels.size())
print(inputs.view(1,6,5))
x = inputs.view(1,6,5)
print(x.size())

torch.Size([6, 5])
torch.Size([6, 5])
torch.Size([6])
tensor([[[1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0.]]])
torch.Size([1, 6, 5])


In [109]:
num_classes = 5
input_size = 5
hidden_size =5
batch_size =1
sequence_length = 6
num_layers = 1

In [131]:
class Model(nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        self.rnn = nn.RNN(input_size = input_size,
                          hidden_size = hidden_size,
                          batch_first = True)
        self.nn = nn.Linear(30,30)
    def forward(self,x,hidden):
        x = x.view(batch_size, sequence_length, input_size)
        
        out, hidden = self.rnn(x, hidden)
    
        out = self.nn(out)
        
        out = out.view(-1,num_classes)
        out = Variable(out, requires_grad=True)
        hidden = Variable(hidden, requires_grad=True)
        #print(out.size())
        return hidden, out
    
    def init_hidden(self):
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))

In [132]:
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= 0.001)


In [133]:
for epoch in range(1000):
    hidden,outputs = model(inputs,hidden)
    print(outputs)
    print(hidden)
    optimizer.zero_grad()
    print(torch.max(outputs, 1)[1])
    #print(outputs)
    print(labels)
    loss = criterion(outputs, labels) # label size 6 / outputs size 6,5
    #print(loss)
    
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch +1, loss.data))
    print("predicted string: ", ''.join(result_str))
    
print("leaning finished")

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 3 1 1, but got 3-dimensional input of size [1, 6, 5] instead

In [113]:
for epoch in range(100):
    
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()
    sys.stdout.write("predicted string: ")
    
    for inpute,label in zip(inputs, labels):
        hidden, output = model(inpute, hidden)
        val, idx = output.max(1)
        sys.stdout.write(idx2char[idx.data[0]])
        loss += criterion(output, label)
    
print(",epoch: %d, loss: %1.3f" % (epoch +1, loss.data[0]))

loss.backward()
optimizer.step()

predicted string: 

RuntimeError: shape '[1, 6, 5]' is invalid for input of size 5